# Proyecto Individual 1

## Parte 1 EDA

Se importan galerias a utilizar

In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import os
from sklearn.metrics.pairwise import linear_kernel


Como primer paso se carga el dataset de reviews y se analiza el dataframe, viendo columnas, nulos, forma, tamaños

In [ ]:
# Se crea una lista en la que guardaremos diccionarios
linea_dicc = []

# Se abre y carga el archivo linea a linea (tiene problemas de forma)
with open('src/data/australian_user_reviews.json', 'r', encoding='utf-8') as a:
    for num_linea, linea in enumerate(a, start=1):
            data = eval(linea)
            linea_dicc.append(data)

# Finalmente se carga los datos a un dataframe y se imprimen los resultados
reviews = pd.DataFrame(linea_dicc)
#Hay 25799 registros, en 3 columnas
reviews.head()

In [ ]:
reviews['reviews'][1]#Se observa la composicion de la columna reviews, que esta anidada

In [ ]:
reviews.isna().sum().sort_values(ascending=False)#No hay datos nulos

Se realiza la carga del dataset de juegos y se analiza el resultado:

In [ ]:
#Se carga el dataframe de los juegos de steam y se imprime el resultado
juegos = pd.read_json('src/data/output_steam_games.json', lines=True)
juegos.head()
#Tiene 120445 datos

In [ ]:
juegos.isna().sum().sort_values(ascending=False) #Hay muchisimos datos nulos

Se carga y analiza el dataset de usuarios y se examina el resultado

In [ ]:
import gzip

In [ ]:
# Se crea una lista en la que guardaremos diccionarios
linea_dicc = []

# Se abre y carga el archivo linea a linea (tiene problemas de forma)
with gzip.open('src/data/users_items.json.gz', 'rt', encoding='utf-8') as a:
    for num_linea, linea in enumerate(a, start=1):
            data = eval(linea)
            linea_dicc.append(data)

# Finalmente se carga los datos a un dataframe y se imprimen los resultados
items = pd.DataFrame(linea_dicc)
items.head()
#88310 filas

In [ ]:
items['items'][1]#Se examina la columna anidada

In [ ]:
items.isna().sum().sort_values(ascending=False) #No hay datos nulos

# Parte 2 ETL

### Reviews

La columna reviews tiene los datos anidados, se procede a separarlos en columnas e integrarlo al dataframe reviews_sinuser, tambien se reemplaza '' por nan

In [ ]:
df_exploded = reviews.explode('reviews') #se separa en filas distintas cada review
df_normalized = pd.json_normalize(df_exploded['reviews'])#se separa en columnas la columna anidada
df_normalized.head()


In [ ]:
df_exploded.reset_index(drop=True, inplace=True)#reseteamos indices
df_normalized.reset_index(drop=True, inplace=True)

In [ ]:
# Combinar con el DataFrame original
reviews_f = pd.concat([df_exploded, df_normalized], axis=1).drop('reviews', axis=1)
# Mostrar el DataFrame resultante
reviews_f.head()

In [ ]:
reviews_f['funny'] = reviews_f['funny'].replace('', np.nan)

In [ ]:
reviews_f['last_edited'] = reviews_f['last_edited'].replace('', np.nan)

In [ ]:
reviews_f.head()#resultado

In [ ]:
reviews_final=reviews_f.drop_duplicates()#elimino duplicados quedando 58459 datos de 59333.
reviews_final.info()

In [ ]:
from textblob import TextBlob #se importa esta libreria para hacer el analisis de sentimineto

In [ ]:
def classify_sentiment(text):
    if isinstance(text, str):  # Comprueba si el valor es una cadena
        analysis = TextBlob(text)
        polarity = analysis.sentiment.polarity

        # Clasificar según la polaridad
        if polarity < -0.1:
            return 0  # Reseña mala
        elif polarity > 0:
            return 2  # Reseña positiva
        else:
            return 1  # Reseña neutra
    else:
        return np.nan  # Devuelve NaN si el valor no es una cadena

# Aplicar la función a la columna 'review' y guardar los resultados en la nueva columna 'sentiment_analysis'
reviews_final['review'] = reviews_final['review'].apply(classify_sentiment)


In [ ]:
reviews_final.head()

In [ ]:
counts = reviews_final['review'].value_counts() #cuento cuantos review de cada categoria hay
print(counts)

In [ ]:
counts2 = reviews_final['recommend'].value_counts() #cuento cuantos review de cada categoria hay: hay mas positivas
print(counts2)

### Juegos

Al dataframe juegos se eliminan nulos, duplicados y ciertas columnas

In [ ]:
juegos.head()

In [ ]:
juegos1=juegos.dropna(how='all') #Se eliminan las filas que no tienen ningun registro, quedando 32135 registros

In [ ]:
juegos1=juegos1.drop_duplicates(subset=['id'])#elimino los juegos duplicados, basandome en el id, solo se eliminan 2 quedando 32133 juegos

In [ ]:
juegos1=juegos1.drop(['title','url','tags','reviews_url','publisher'], axis=1)

In [ ]:
juegos1.info()

### items

Al data frame items se le eliminan duplicados y algunas columnas, posteriormente se desanida la columna items

In [ ]:
items1=items.drop_duplicates(subset=['user_id'])#elimino los usuarios duplicados, basandome en el id
##quedan 87626 datos de 88310

In [ ]:
items1=items1.drop(['user_url','steam_id'],axis=1)
items1.head()

In [ ]:
df_list = []

# Iterar a través de cada fila del DataFrame y aplicar el proceso de normalización
for index, row in items1.iterrows():
    df_normalized = pd.json_normalize(row['items'])
    if not df_normalized.empty:
        df_combined = pd.concat([pd.DataFrame(row).transpose()] * len(df_normalized), ignore_index=True)
        df_combined = pd.concat([df_combined, df_normalized], axis=1)
        df_list.append(df_combined)

# Combinar los DataFrames resultantes en uno solo si hay elementos en df_list
if df_list:
    df_final = pd.concat(df_list, ignore_index=True)
    # Mostrar el DataFrame resultante
    print(df_final)
else:
    print("No se generaron DataFrames para concatenar.")



In [ ]:
items_fin=df_final.drop('items',axis=1)

In [ ]:
items_fin.head()

In [ ]:
items_fin_sin_0= items_fin[items_fin['playtime_forever'] != 0]#se eliminan las filas en que el usuario no jugo 
#quedan 3246352 de 5094082 

In [ ]:
items_fin_sin_0.reset_index(drop=True, inplace=True)


In [ ]:
juegos1.reset_index(drop=True, inplace=True)

In [ ]:
reviews_final.reset_index(drop=True, inplace=True)

## Parte 3 tratamiento outliers

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.distplot(items_fin_sin_0['playtime_forever'], hist=True, kde=True, rug=True) #Se observa la distribucion y se ve que claramente muchos juegos se han jugado poco


In [ ]:
plt.figure(figsize=(8, 6))
#En el boxplot se pueden observar los cuartiles, pero en este caso debido.. 
#a la gran aglomeracion en las pocas horas jugadas hace que el boxplot se vea como una linea
sns.boxplot(x=items_fin_sin_0['playtime_forever'])
plt.title('Diagrama de caja para observar outliers')
plt.show()

In [ ]:
Q1 = items_fin_sin_0['playtime_forever'].quantile(0.25)
Q3 = items_fin_sin_0['playtime_forever'].quantile(0.75)
IQR = Q3 - Q1
print(Q1,Q3,IQR)

In [ ]:
# Definir límites para detectar outliers
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5* IQR
print (lower_limit, upper_limit)
#El resultado de los limites (-1102,1954) se los va a modificar, debido a que el minimo de hs jugadas debe ser 0 y el maximo de hs jugadas podria ser mayor
#Se utilizara minimo 0, max=5hs/dia*60min*365dias=109500  (se toma un valor que considero extremo[4hs/dia durante un año])

In [ ]:
# Filtrar outliers
lower_limit=0
upper_limit=109500
items_fin_filtered = items_fin_sin_0[(items_fin_sin_0['playtime_forever'] > lower_limit) & (items_fin_sin_0['playtime_forever'] < upper_limit)]

In [ ]:
#se desanida columna gebre
genero=juegos1.explode('genres')
genero=genero.explode('specs')
genero = genero.rename(columns={'id': 'item_id'})
genero.head()

## Parte 4 Funciones

### Funcion 1

Debe devolver año con mas horas jugadas para dicho género.
Primero se crean los df necesarios y se los une, luego se crea la funcion 

In [ ]:
funcion1i=['item_id','playtime_forever']
funcion1g=['genres','release_date','item_id']
df1=items_fin_sin_0[funcion1i]
df11=genero[funcion1g]

In [ ]:
df1['item_id'] = pd.to_numeric(df1['item_id'], errors='coerce').fillna(0).astype(int)
df11['item_id'] = pd.to_numeric(df11['item_id'], errors='coerce').fillna(0).astype(int)
#por problemas en el formato de la columna item_id, hay que pasarlo a int para lograr el merge

In [ ]:
# Fusionar DataFrames después de convertir los tipos de datos
funcion_1 = pd.merge(df1, df11, on='item_id')

In [ ]:
import re
# Crear una función para extraer el año 
def obtener_anio(fecha):
    if fecha is not None:
        match = re.search(r'\d{4}', str(fecha))  # Convierte fecha a cadena y luego busca el patrón de cuatro dígitos que represente el año
        if match:
            return match.group()
        else:
            return 'Formato de fecha no válido'
    else:
        return 'Valor nulo'

# Aplicar la función a la columna 'fecha' y almacenar los resultados en una nueva columna 'anio'
funcion_1['release_date'] = funcion_1['release_date'].apply(obtener_anio)

In [ ]:
funcion_1.reset_index(drop=True, inplace=True)

In [ ]:
def PlayTimeGenre(genero):
    data_genero = funcion_1[funcion_1['genres'] == genero]  # Filtrar el DataFrame por el género especificado
    data_genero_agrupado = data_genero.groupby('release_date')['playtime_forever'].sum()  # Calcular el tiempo total jugado por año
    anio_max_tiempo = data_genero_agrupado.idxmax()  # Encontrar el año con el tiempo máximo
    return anio_max_tiempo

In [ ]:
a=PlayTimeGenre('Action')
a

### Funcion 2

Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Primero se crean los df necesarios y se los une, finalmente se crea la funcion

In [ ]:
juegos1.head()

In [ ]:
#Hago una transformacion del formato fecha
reviews_final['posted'] = reviews_final['posted'].apply(obtener_anio)
reviews_final = reviews_final[reviews_final['posted'] != 'Formato de fecha no válido'] 
reviews_final['posted'] = pd.to_datetime(reviews_final['posted']) 



#Hago una transformacion del formato de item_id
genero['item_id'] = pd.to_numeric(genero['item_id'], errors='coerce').fillna(0).astype(int)
reviews_final['item_id'] = pd.to_numeric(reviews_final['item_id'], errors='coerce').fillna(0).astype(int)
items_fin_filtered['item_id'] = pd.to_numeric(items_fin_filtered['item_id'], errors='coerce').fillna(0).astype(int)
juegos1.rename(columns={'id': 'item_id'}, inplace=True)
juegos1['item_id'] = pd.to_numeric(juegos1['item_id'], errors='coerce').fillna(0).astype(int)

In [ ]:
#Genero los df que necesito
funcion2g=['genres','item_id']
funcion2r=['posted','user_id']
funcion2i=['user_id','item_id','playtime_forever']
dfg=genero[funcion2g]
dfr=reviews_final[funcion2r]
dfi=items_fin_filtered[funcion2i]

In [ ]:
# Unir los dos primeros dataframes
merged_df_intermediate = dfr.merge(dfi, on='user_id')
# Unir el dataframe resultante con el tercer dataframe
funcion_2 = merged_df_intermediate.merge(dfg, on='item_id')
# Mostrar el dataframe resultante
funcion_2.info()

In [ ]:
#Elimino duplicados y los nulos
df_no_duplicates = funcion_2.drop_duplicates(subset=['user_id', 'item_id', 'playtime_forever', 'genres'], keep='first')
funcion2= df_no_duplicates.dropna(subset=['posted'])

In [ ]:
funcion2.reset_index(drop=True, inplace=True)

In [ ]:
def UserForGenre(genero):
    funcion2['posted'] = pd.to_datetime(funcion2['posted'])  # Convertir 'posted' a tipo datetime
    df_filtered = funcion2[funcion2['genres'] == genero]
    max_hours_user = df_filtered.groupby('user_id')['playtime_forever'].sum().idxmax()
    acumulacion_por_anio = df_filtered.groupby(df_filtered['posted'].dt.year)['playtime_forever'].sum().tolist()
    return max_hours_user, acumulacion_por_anio

In [ ]:
# Pruebo la funcion
genero_dado = 'Adventure'
resultado = UserForGenre(genero_dado)

## Funcion 3

Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [ ]:
#Genero los df que necesito
funcion3r=['posted','item_id','recommend','review']
funcion3j=['app_name','item_id']
df3r=reviews_final[funcion3r]
df3j=juegos1[funcion3j]


In [ ]:
funcion_3 = df3r.merge(df3j, on='item_id')
funcion_3['score'] = ''
# Mostrar el dataframe resultante
funcion_3.head()

In [ ]:
def UsersRecommend(year):
    funcion_3['score'] = 0
    #Se realiza un sistema de puntajes para evitar el sesgo por popularidad de un juego, el cual tendria muchos
    #positivos pero probablemente muchos negativos, pudiendo aparecer en los mas recomendados y en los menos al mismo tiempo
    for index, row in funcion_3.iterrows():
        if row['recommend']:
            funcion_3.at[index, 'score'] += 1
        else:
            funcion_3.at[index, 'score'] -= 1
        if row['review'] == 2:
            funcion_3.at[index, 'score'] += 1
        elif row['review'] == 0:
            funcion_3.at[index, 'score'] -= 1
    funcion_3_filtered = funcion_3[funcion_3['posted'].dt.year == year]
    ranking = funcion_3_filtered.groupby('app_name')['score'].sum().nlargest(3).reset_index()
    return  ranking

In [ ]:
#Prueba
resultado = UsersRecommend(2015)
resultado

## Funcion 4

Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

In [ ]:
def UsersNotRecommend(year):
    funcion_3['score'] = 0
    #Se realiza un sistema de puntajes para evitar el sesgo por popularidad de un juego, el cual tendria muchos
    #positivos pero probablemente muchos negativos, pudiendo aparecer en los mas recomendados y en los menos al mismo tiempo
    for index, row in funcion_3.iterrows():
        if row['recommend']:
            funcion_3.at[index, 'score'] += 1
        else:
            funcion_3.at[index, 'score'] -= 1
        if row['review'] == 2:
            funcion_3.at[index, 'score'] += 1
        elif row['review'] == 0:
            funcion_3.at[index, 'score'] -= 1
    funcion_3['posted'] = pd.to_datetime(funcion_3['posted'])
    funcion_4_filtered = funcion_3[funcion_3['posted'].dt.year == year]
    ranking = funcion_4_filtered.groupby('app_name')['score'].sum().nsmallest(3).reset_index()
    return  ranking

In [ ]:
r=UsersNotRecommend(2015)
r

## Funcion 5


Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [ ]:
#Se realiza el df con las columnas posted y review. Se utiliza posted y no release data por que esta mas relacionado
# a la review en concreto a la que se le hace el sentimentanalysis
df5=['posted','review']
funcion_5=reviews_final[df5]

In [ ]:
funcion_5.head()

In [ ]:
#Se formula la funcion inicializando variables y sumando segun sea el caso
def sentiment_analysis( año : int ): 
    Positivo=0
    Negativo=0
    Neutro=0
    funcion_5['posted'] = funcion_5['posted'].str.split('-').str[0]
    funcion_5_filtered = funcion_5[funcion_5['posted'] == año]
    for index, row in funcion_5_filtered.iterrows():
         if row['review'] == 2:
            Positivo+=1
         elif row['review'] == 0:
            Negativo+=1
         else:
            Neutro+=1
    return{'Positivos': Positivo,' Negativos': Negativo, ' Neutros': Neutro}

In [ ]:
def sentiment_analysis(año: int): 
    Positivo = 0
    Negativo = 0
    Neutro = 0
    funcion_5['posted'] = pd.to_datetime(funcion_5['posted'])
    funcion_5_filtered = funcion_5[funcion_5['posted'].dt.year == año]
    for index, row in funcion_5_filtered.iterrows():
         if row['review'] == 2:
            Positivo += 1
         elif row['review'] == 0:
            Negativo += 1
         else:
            Neutro += 1
    return {'Positivos': Positivo, 'Negativos': Negativo, 'Neutros': Neutro}

In [ ]:
a=sentiment_analysis(2014)
a

## Parte 5 Crear parquets


Armo parquets's con posibles resultados de las funciones

In [ ]:
#Armo dataframe con resultados y lo paso a csv
#Armo df para aplicar funcion, eliminando none y accounting porq no tienen datos
gen = genero['genres'].unique()
gen = [x for x in gen if x!=None]
gen = [x for x in gen if x!='Accounting']
df_genero = pd.DataFrame(gen, columns=['Género'])
#Aplico funcion
df_genero['Año resultado'] = df_genero['Género'].apply(PlayTimeGenre)
#Exporto
ruta_carpeta = r'C:\Users\nicoi\Desktop\HENRY\Henry data\Fast Api\datas'
nombre_archivo = 'f1.parquet'
ruta_completa = ruta_carpeta + nombre_archivo
df_genero.to_parquet(ruta_completa, index=False)

In [ ]:
df_genero.head()

In [ ]:
#Armo dataframe con resultados y lo paso a csv
#Armo df para aplicar funcion, eliminando none y accounting porq no tienen datos
gen = genero['genres'].unique()
gen = [x for x in gen if x!=None]
gen = [x for x in gen if x!='Accounting']
df_generos = pd.DataFrame(gen, columns=['Género'])
#Aplico funcion
df_generos['Usuario'] = df_generos['Género'].apply(PlayTimeGenre)
#Aplico funcion
df_generos[['usuario', 'hs por año']] = df_generos['Género'].apply(UserForGenre).apply(pd.Series)
#Exporto
ruta_carpeta = r'C:\Users\nicoi\Desktop\HENRY\Henry data\Fast Api\datas'
nombre_archivo = 'f2.parquet'
ruta_completa = ruta_carpeta + nombre_archivo
df_generos.to_parquet(ruta_completa, index=False)

In [ ]:
df_generos.head()

In [ ]:
#Se genera df de resultados
años = funcion_3['posted'].unique()
# Convertir la lista de fechas a una serie de Pandas
serie_fechas = pd.Series(años)
# Convertir la serie de Pandas a tipo datetime y extraer los años
anios = pd.to_datetime(serie_fechas).dt.year
# Convertir la serie de años en un DataFrame
df_anios = pd.DataFrame(anios, columns=['posted'])
df_anios = df_anios.sort_values('posted')
df_anios = df_anios.reset_index(drop=True)
#Se crea el df
dataframes = []
for index, row in df_anios.iterrows():
    year = row['posted']
    df_resultado = UsersRecommend(year)
    df_resultado['Año'] = year  # Agregar la columna 'Año'
    dataframes.append(df_resultado)

# Combinar todos los DataFrames en uno solo
f3 = pd.concat(dataframes, ignore_index=True)
#Exporto
ruta_carpeta = r'C:\Users\nicoi\Desktop\HENRY\Henry data\Fast Api\datas'
nombre_archivo = 'f3.parquet'
ruta_completa = ruta_carpeta + nombre_archivo
f3.to_parquet(ruta_completa, index=False)

In [ ]:
f3.head()

In [ ]:
#Se crea el df
dataframes = []
for index, row in df_anios.iterrows():
    year = row['posted']
    df_resultado = UsersNotRecommend(year)
    df_resultado['Año'] = year  # Agregar la columna 'Año'
    dataframes.append(df_resultado)

# Combinar todos los DataFrames en uno solo
f4 = pd.concat(dataframes, ignore_index=True)
#Exporto
ruta_carpeta = r'C:\Users\nicoi\Desktop\HENRY\Henry data\Fast Api\datas'
nombre_archivo = 'f4.parquet'
ruta_completa = ruta_carpeta + nombre_archivo
f4.to_parquet(ruta_completa, index=False)

In [ ]:
f4.head()

In [ ]:
dataframes = []
for index, row in df_anios.iterrows():
    year = row['posted']
    df_resultado = sentiment_analysis(year)
    df_resultado['Año'] = year  # Agregar la columna 'Año'
    dataframes.append(df_resultado)
    f5=pd.DataFrame(dataframes)
#Exporto
ruta_carpeta = r'C:\Users\nicoi\Desktop\HENRY\Henry data\Fast Api\datas'
nombre_archivo = 'f5.parquet'
ruta_completa = ruta_carpeta + nombre_archivo
f5.to_parquet(ruta_completa, index=False)
    

## Parte 6: Modelo de Machine Learning

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [117]:
#Se eligen las columnas consideradas posiblemente utiles y se arman dfs
rf=['user_id','funny','posted','item_id','helpful','recommend','review']
iff=['user_id','item_id','item_name','playtime_forever']
j=['genres','app_name','release_date','price','item_id','developer','specs']
reviewf=reviews_final[rf]
itemsf=items_fin_filtered[iff]
juegosf=juegos1[j]
merged_df = pd.merge(reviewf, itemsf, on=['user_id', 'item_id'])
merged_df = pd.merge(merged_df, juegosf, on='item_id')

In [118]:
counts = merged_df['helpful'].value_counts()#Mas de la mitad de los datos 'no rating yet'.
counts

helpful
No ratings yet                                      16919
1 of 1 people (100%) found this review helpful       3829
0 of 1 people (0%) found this review helpful         2170
1 of 2 people (50%) found this review helpful        1383
2 of 2 people (100%) found this review helpful       1076
                                                    ...  
91 of 128 people (71%) found this review helpful        1
3 of 37 people (8%) found this review helpful           1
12 of 41 people (29%) found this review helpful         1
37 of 45 people (82%) found this review helpful         1
21 of 34 people (62%) found this review helpful         1
Name: count, Length: 874, dtype: int64

In [119]:
counts1 = genero['genres'].value_counts()#evaluo esta columna para usarla en el momento de achicar el dataframe data
counts1

genres
Indie                        67958
Action                       56076
Casual                       35888
Simulation                   34355
Strategy                     32239
Adventure                    31884
RPG                          24019
Free to Play                 10183
Sports                        7537
Massively Multiplayer         6194
Racing                        5972
Early Access                  5613
Design &amp; Illustration     1110
Web Publishing                 756
Utilities                      750
Animation &amp; Modeling       410
Education                      287
Software Training              197
Photo Editing                  183
Video Production               178
Audio Production               148
Accounting                       7
Name: count, dtype: int64

In [120]:
merged_df.head() #item_name y app_name es lo mismo

,user_id,funny,posted,item_id,helpful,recommend,review,item_name,playtime_forever,genres,app_name,release_date,price,developer,specs
0,76561197970982479,NaN,2011-01-01,1250,No ratings yet,True,2.0,Killing Floor,10006,[Action],Killing Floor,2009-05-14,19.99,Tripwire Interactive,"[Single-player, Multi-player, Co-op, Cross-Pla..."
1,DJKamBer,NaN,2013-01-01,1250,No ratings yet,True,2.0,Killing Floor,2851,[Action],Killing Floor,2009-05-14,19.99,Tripwire Interactive,"[Single-player, Multi-player, Co-op, Cross-Pla..."
2,diego9031,NaN,2015-01-01,1250,No ratings yet,True,1.0,Killing Floor,1204,[Action],Killing Floor,2009-05-14,19.99,Tripwire Interactive,"[Single-player, Multi-player, Co-op, Cross-Pla..."
3,GamerFag,NaN,2010-01-01,1250,No ratings yet,True,2.0,Killing Floor,33723,[Action],Killing Floor,2009-05-14,19.99,Tripwire Interactive,"[Single-player, Multi-player, Co-op, Cross-Pla..."
4,76561198092227136,NaN,2014-01-01,1250,No ratings yet,True,1.0,Killing Floor,8694,[Action],Killing Floor,2009-05-14,19.99,Tripwire Interactive,"[Single-player, Multi-player, Co-op, Cross-Pla..."


In [121]:
merged_df.info()#Se observan pocos datos en funny.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32535 entries, 0 to 32534
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           32535 non-null  object        
 1   funny             4007 non-null   object        
 2   posted            32535 non-null  datetime64[ns]
 3   item_id           32535 non-null  int32         
 4   helpful           32535 non-null  object        
 5   recommend         32535 non-null  object        
 6   review            32535 non-null  float64       
 7   item_name         32535 non-null  object        
 8   playtime_forever  32535 non-null  int64         
 9   genres            31221 non-null  object        
 10  app_name          32535 non-null  object        
 11  release_date      31233 non-null  object        
 12  price             32269 non-null  object        
 13  developer         31155 non-null  object        
 14  specs             3249

In [122]:
#Se eliminan columnas innecesarias
a=['user_id','recommend','review','item_id','app_name','genres','release_date','posted','playtime_forever']
b=['genres','item_id','app_name','specs','developer']
data= merged_df[a]
datb=genero[b]
data.head()

,user_id,recommend,review,item_id,app_name,genres,release_date,posted,playtime_forever
0,76561197970982479,True,2.0,1250,Killing Floor,[Action],2009-05-14,2011-01-01,10006
1,DJKamBer,True,2.0,1250,Killing Floor,[Action],2009-05-14,2013-01-01,2851
2,diego9031,True,1.0,1250,Killing Floor,[Action],2009-05-14,2015-01-01,1204
3,GamerFag,True,2.0,1250,Killing Floor,[Action],2009-05-14,2010-01-01,33723
4,76561198092227136,True,1.0,1250,Killing Floor,[Action],2009-05-14,2014-01-01,8694


In [123]:
counts2 = data['item_id'].value_counts()#Hay 2150 juegos
counts2.info()

<class 'pandas.core.series.Series'>
Index: 2150 entries, 730 to 307130
Series name: count
Non-Null Count  Dtype
--------------  -----
2150 non-null   int64
dtypes: int64(1)
memory usage: 25.2 KB


In [124]:
datb.info() #Hay generos vacios

<class 'pandas.core.frame.DataFrame'>
Index: 338027 entries, 0 to 32132
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   genres     321944 non-null  object
 1   item_id    338027 non-null  int32 
 2   app_name   338022 non-null  object
 3   specs      337086 non-null  object
 4   developer  320839 non-null  object
dtypes: int32(1), object(4)
memory usage: 14.2+ MB


In [125]:
datb=datb.dropna()
datb = datb.drop_duplicates()

In [126]:
#Se crea una funcion para agregar una columna score
def UsersRecommendAllYears(data):
    data['score'] = 0
    # Se realiza un sistema de puntajes para evitar el sesgo por popularidad de un juego
    for index, row in data.iterrows():
        if row['recommend']:
            data.at[index, 'score'] += 1
        else:
            data.at[index, 'score'] -= 1
        if row['review'] == 2:
            data.at[index, 'score'] += 1
        elif row['review']==0:
            data.at[index, 'score'] -= 1
    # Agrupar por 'item_id' y sumar los valores de 'score'
    data = data.groupby('item_id')['score'].sum().reset_index()
    return data

In [127]:
#Se aplica la funcion y se suma la columna score al dataframe
data2=UsersRecommendAllYears(data)
data_final = pd.merge(data, data2, on='item_id')

C:\TEMP\ipykernel_9668\3073876890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['score'] = 0


In [128]:
#Se elimina score x porq era un puntaje parcial, solo quiero el final para filtrar mas tarde
data_final =data_final.drop(columns=['score_x'])
#Se elimina app_name porq lo tengo en datb
data_final =data_final.drop(columns=['app_name'])
#Renombro columna
data_final =data_final.rename(columns={'score_y': 'score'})


In [129]:
data_final.head()

,user_id,recommend,review,item_id,genres,release_date,posted,playtime_forever,score
0,76561197970982479,True,2.0,1250,[Action],2009-05-14,2011-01-01,10006,336
1,DJKamBer,True,2.0,1250,[Action],2009-05-14,2013-01-01,2851,336
2,diego9031,True,1.0,1250,[Action],2009-05-14,2015-01-01,1204,336
3,GamerFag,True,2.0,1250,[Action],2009-05-14,2010-01-01,33723,336
4,76561198092227136,True,1.0,1250,[Action],2009-05-14,2014-01-01,8694,336


In [130]:
ruta_carpeta = r'C:\Users\nicoi\Desktop\HENRY\Henry data\Fast Api\datas\\'  # Asegúrate de incluir una barra diagonal adicional al final
nombre_archivo = 'ml1.parquet'
ruta_completa = os.path.join(ruta_carpeta, nombre_archivo)  # Usa os.path.join para manejar las rutas de forma segura
# Guardar el DataFrame en formato Parquet
data_final.to_parquet(ruta_completa, index=False)

In [131]:
ruta_carpeta = r'C:\Users\nicoi\Desktop\HENRY\Henry data\Fast Api\datas\\'  # Asegúrate de incluir una barra diagonal adicional al final
nombre_archivo = 'ml2.parquet'
ruta_completa = os.path.join(ruta_carpeta, nombre_archivo)  # Usa os.path.join para manejar las rutas de forma segura
# Guardar el DataFrame en formato Parquet
datb.to_parquet(ruta_completa, index=False)

In [132]:
data_final=pd.read_parquet('src/datas/ml1.parquet')
datb=pd.read_parquet('src/datas/ml2.parquet')


In [133]:
data_final.head()

,user_id,recommend,review,item_id,genres,release_date,posted,playtime_forever,score
0,76561197970982479,True,2.0,1250,[Action],2009-05-14,2011-01-01,10006,336
1,DJKamBer,True,2.0,1250,[Action],2009-05-14,2013-01-01,2851,336
2,diego9031,True,1.0,1250,[Action],2009-05-14,2015-01-01,1204,336
3,GamerFag,True,2.0,1250,[Action],2009-05-14,2010-01-01,33723,336
4,76561198092227136,True,1.0,1250,[Action],2009-05-14,2014-01-01,8694,336


In [134]:
datb.head()

,genres,item_id,app_name,specs,developer
0,Action,761140,Lost Summoner Kitty,Single-player,Kotoshiro
1,Casual,761140,Lost Summoner Kitty,Single-player,Kotoshiro
2,Indie,761140,Lost Summoner Kitty,Single-player,Kotoshiro
3,Simulation,761140,Lost Summoner Kitty,Single-player,Kotoshiro
4,Strategy,761140,Lost Summoner Kitty,Single-player,Kotoshiro


In [135]:
def extract_genres_by_item_id(item_id):
    #Primero se realiza una serie de filtros para achicar el df

    #Se extrae el o los generos asociados al juego usando datb
    genres = datb[datb['item_id'] == item_id]['genres'].tolist()
    #Se extraen los item_id relacionados con los genereos del juego en cuestion
    related_items1 = datb[datb['genres'].apply(lambda x: any(item in x for item in genres))]['item_id'].unique().tolist()
    #Se extrae el o los specs asociados al juego usando datb
    genres = datb[datb['item_id'] == item_id]['specs'].tolist()
    #Se extraen los item_id relacionados con los specs del juego en cuestion
    related_items2 = datb[datb['specs'].apply(lambda x: any(item in x for item in genres))]['item_id'].unique().tolist()
    #Finalmente se sacan los item_id que aparecen en los 3
    common_items = set(related_items1).intersection(related_items2)

    #Se extrae de data final solo los item_id identificados anteriormente, se toma aquellas filas que son recomendadas 
    #y se eliminan las columnas recommend y review por ser irrelevantes luego del filtro
    filtered_data_final = data_final[data_final['item_id'].isin(common_items)]
    filtered_data_final = filtered_data_final[(filtered_data_final['recommend'] == True) & (filtered_data_final['review'] == 2)]
    filtered_data_final = filtered_data_final.drop(columns=['recommend', 'review'])

    # Ordenar el DataFrame por 'score' en orden descendente
    filtered_data_final.sort_values('score', ascending=False, inplace=True)
    # Obtener los 5 item_id con la puntuación más alta
    top_5_items = filtered_data_final['item_id'].unique()[:5]
    app_names = list(set(datb[datb['item_id'].isin(top_5_items)]['app_name']))
    
    return app_names

In [136]:
aux=extract_genres_by_item_id(10)
aux

['Rust',
 'Unturned',
 'Counter-Strike: Global Offensive',
 'Terraria',
 'Left 4 Dead 2']